In [ ]:
from skyfield.api import load

# Load planetary data and time
planets = load('de421.bsp')
ts = load.timescale()
t = ts.now()

# Get Earth and Mars
earth = planets['earth']
mars = planets['mars']

# Observe Mars from Earth
astrometric = earth.at(t).observe(mars)
ra, dec, distance = astrometric.radec()

# Print results
print(f"Mars Right Ascension: {ra}")
print(f"Mars Declination: {dec}")
print(f"Distance (AU): {distance}")


In [ ]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QLabel, QComboBox
from PyQt5.QtCore import QTimer
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure
from skyfield.api import load

# Load planetary data
planets = load('de421.bsp')
ts = load.timescale()

# Only include planets actually in the kernel
planet_names = {
    'Mercury': 'mercury',
    'Venus': 'venus',
    'Earth': 'earth',
    'Mars': 'mars',
    'Jupiter': 'jupiter barycenter',
    'Saturn': 'saturn barycenter',
    'Uranus': 'uranus barycenter',
    'Neptune': 'neptune barycenter',
    'Pluto': 'pluto barycenter'
}

class PlanetTracker(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Real-Time Planet Tracker")
        self.setGeometry(100, 100, 600, 600)

        layout = QVBoxLayout()

        # Dropdown to select planet
        self.dropdown = QComboBox()
        self.dropdown.addItems(planet_names.keys())
        self.dropdown.currentIndexChanged.connect(self.update_data)

        # Info labels
        self.ra_label = QLabel("Right Ascension: ")
        self.dec_label = QLabel("Declination: ")
        self.dist_label = QLabel("Distance (AU): ")

        # Matplotlib plot
        self.figure = Figure(figsize=(5, 5))
        self.canvas = FigureCanvas(self.figure)
        self.ax = self.figure.add_subplot(111)
        self.ax.set_title("Planet Position (relative)")
        self.ax.set_xlim(-2, 2)
        self.ax.set_ylim(-2, 2)
        self.ax.grid(True)

        layout.addWidget(self.dropdown)
        layout.addWidget(self.ra_label)
        layout.addWidget(self.dec_label)
        layout.addWidget(self.dist_label)
        layout.addWidget(self.canvas)

        self.setLayout(layout)

        # Timer for updates
        self.timer = QTimer()
        self.timer.timeout.connect(self.update_data)
        self.timer.start(1000)

        self.update_data()

    def update_data(self):
        planet_name = self.dropdown.currentText()
        skyfield_name = planet_names[planet_name]
        t = ts.now()
        earth = planets['earth']
        target = planets[skyfield_name]

        astrometric = earth.at(t).observe(target)
        ra, dec, distance = astrometric.radec()
        pos = astrometric.position.au  # x, y, z in AU

        # Update labels
        self.ra_label.setText(f"Right Ascension: {ra}")
        self.dec_label.setText(f"Declination: {dec}")
        self.dist_label.setText(f"Distance (AU): {distance.au:.4f}")

        # Update plot
        self.ax.clear()
        self.ax.set_title("Planet Position (relative to Earth)")

        # Calculate limit based on distance with some margin
        limit = max(2, abs(pos[0]) * 1.2, abs(pos[1]) * 1.2)

        self.ax.set_xlim(-limit, limit)
        self.ax.set_ylim(-limit, limit)

        self.ax.grid(True)
        self.ax.plot(0, 0, 'bo', label='Earth')
        self.ax.plot(pos[0], pos[1], 'ro', label=planet_name)
        self.ax.legend()
        self.canvas.draw()
        
if __name__ == "__main__":
    app = QApplication(sys.argv)
    tracker = PlanetTracker()
    tracker.show()
    sys.exit(app.exec_())
